<a href="https://colab.research.google.com/github/ncdelvalle/The-SINAYA-Project/blob/main/FINAL_SINAYA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow numpy pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from tensorflow.keras.metrics import Accuracy, MeanSquaredError
from sklearn.preprocessing import MinMaxScaler

# Load the training data from a CSV file
data = pd.read_csv("/content/Training Dataset v2.csv")

# Normalize the input and output data
scaler = MinMaxScaler()
X = scaler.fit_transform(data[["pH", "do", "temp"]])
y = scaler.fit_transform(data[["ammonia"]])

# Define the input layers
input_pH = tf.keras.layers.Input(shape=(1,))
input_do = tf.keras.layers.Input(shape=(1,))
input_temp = tf.keras.layers.Input(shape=(1,))

# Define the fuzzy membership functions
pH = tf.keras.layers.Lambda(lambda x: tf.clip_by_value(tf.pow(tf.subtract(tf.divide(x, 6.96), 0.765), 3), 0, 1))(input_pH)
do = tf.keras.layers.Lambda(lambda x: tf.clip_by_value(tf.pow(tf.divide(tf.subtract(x, 0.26), 7.3), 2), 0, 1))(input_do)
temp = tf.keras.layers.Lambda(lambda x: tf.clip_by_value(tf.divide(tf.subtract(x, 29.06), 3.27), 0, 1))(input_temp)

# Concatenate the fuzzy membership functions
concat = tf.keras.layers.Concatenate()([pH, do, temp])

# Define the fuzzy rule base
rule1 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], x[1]))([pH, do])
rule2 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], x[1]))([do, temp])
rule3 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], x[1]))([pH, temp])
rule4 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], tf.minimum(x[1], x[2])))([pH, do, temp])
rule5 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], x[1]))([pH, tf.subtract(1.0, do)])
rule6 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], tf.subtract(1.0, tf.clip_by_value(x[1], 0, 1))))([tf.subtract(1.0, pH), temp])
rule7 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], tf.subtract(1.0, x[1])))([do, tf.subtract(1.0, pH)])
rule8 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], tf.subtract(1.0, x[1])))([temp, pH])

# Concatenate the rules

# Concatenate the rules
rules = tf.keras.layers.Concatenate()([rule1, rule2, rule3, rule4])

# Define the output layer
output = tf.keras.layers.Dense(1, activation='linear')(rules)

# Define the neural network model
model = tf.keras.models.Model(inputs=[input_pH, input_do, input_temp], outputs=output)

# Train the model
model = Sequential()
model.add(Dense(10, input_shape=(3,), activation='relu'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='nadam', loss='mse', metrics=['accuracy'])
optimizer = optimizers.Nadam(learning_rate=0.001, beta_1=0.6, beta_2=0.665, epsilon=7e-08)
model.fit(x=X, y=y, epochs=3000, batch_size=64)
# Save the model
model.save('SINAYA_FFNN_FINAL4.h5')

Streaming output truncated to the last 5000 lines.
8/8 [==============================] - 0s 4ms/step - loss: 0.0044 - accuracy: 0.0020
Epoch 502/3000
8/8 [==============================] - 0s 4ms/step - loss: 0.0044 - accuracy: 0.0020
Epoch 503/3000
8/8 [==============================] - 0s 4ms/step - loss: 0.0044 - accuracy: 0.0020
Epoch 504/3000
8/8 [==============================] - 0s 4ms/step - loss: 0.0044 - accuracy: 0.0020
Epoch 505/3000
8/8 [==============================] - 0s 4ms/step - loss: 0.0044 - accuracy: 0.0020
Epoch 506/3000
8/8 [==============================] - 0s 4ms/step - loss: 0.0044 - accuracy: 0.0020
Epoch 507/3000
8/8 [==============================] - 0s 4ms/step - loss: 0.0044 - accuracy: 0.0020
Epoch 508/3000
8/8 [==============================] - 0s 4ms/step - loss: 0.0044 - accuracy: 0.0020
Epoch 509/3000
8/8 [==============================] - 0s 4ms/step - loss: 0.0044 - accuracy: 0.0020
Epoch 510/3000
8/8 [==============================] - 0s 3ms/ste

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

# Evaluate the model on the training set
mse, accuracy = model.evaluate(X, y)

y_pred = model.predict(X)
r2 = r2_score(y, y_pred)
mae = mean_absolute_error(y, y_pred)
print("R^2:", r2)
print("MAE:", mae)
print("MSE on training set:", mse)
print('Accuracy on test set:', accuracy)

# Load the validation data from a CSV file
data2 = pd.read_csv("/content/Validation Dataset v2.csv")

# Normalize the input
X = scaler.fit_transform(data[["pH", "do", "temp"]])
x1 = data2[["pH", "do", "temp"]]
input_data = scaler.transform(x1)

# Feed the input data to the model and get the output
output = model.predict(input_data)

# Do something with the output
output = output * (7.44 - 0.16) + 0.16
print(output)

16/16 [==============================] - 0s 3ms/step
R^2: 0.47118579302920816
MAE: 0.03648538055925307
MSE on training set: 0.004058304708451033
Accuracy on test set: 0.0020325202494859695
2/2 [==============================] - 0s 5ms/step
[[2.024127 ]
 [1.9285896]
 [2.0526006]
 [2.0411386]
 [1.7940632]
 [1.3988665]
 [1.3575414]
 [1.8760504]
 [1.6566703]
 [1.647561 ]
 [1.5757756]
 [1.7696714]
 [1.860857 ]
 [1.5480124]
 [1.6985722]
 [2.0281093]
 [2.2818172]
 [2.3242953]
 [2.0534341]
 [2.5135987]
 [2.0402117]
 [2.6179302]
 [2.585473 ]
 [2.5804193]
 [2.5167255]
 [2.4478106]
 [2.3497856]
 [2.347867 ]
 [2.190291 ]
 [2.4077919]
 [2.4252932]
 [2.3142803]
 [2.1997309]
 [2.0538323]
 [2.148562 ]
 [2.2992134]
 [1.5827105]
 [1.615785 ]
 [1.5723549]
 [1.8042231]
 [1.7113307]
 [1.6750814]
 [1.6318911]
 [1.6734807]
 [1.6975524]
 [1.5167032]
 [1.511645 ]
 [1.5217142]
 [1.5395398]
 [1.5722619]]


In [ ]:
from tensorflow.keras.models import load_model
# Load the saved model
loaded_model = load_model('SINAYA_FFNN_FINAL.h5')

In [ ]:
# Generate predictions on new data
y_pred = model.predict(X)

# Compare predictions to actual values
for i in range(len(y)):
    print("Actual:", y[i], "Predicted:", y_pred[i])
    y1 = y[i] * (7.44 - 0.16) + 0.16
    y2 = y_pred[i] * (7.44 - 0.16) + 0.16
    print(y1, y2)

16/16 [==============================] - 0s 1ms/step
Actual: [0.24175824] Predicted: [0.26366702]
[1.92] [2.079496]
Actual: [0.23901099] Predicted: [0.25507364]
[1.9] [2.016936]
Actual: [0.22802198] Predicted: [0.24986175]
[1.82] [1.9789935]
Actual: [0.24038462] Predicted: [0.25905395]
[1.91] [2.0459127]
Actual: [0.24175824] Predicted: [0.25665534]
[1.92] [2.028451]
Actual: [0.24175824] Predicted: [0.26753417]
[1.92] [2.1076488]
Actual: [0.24038462] Predicted: [0.2679027]
[1.91] [2.1103318]
Actual: [0.24038462] Predicted: [0.26710966]
[1.91] [2.1045585]
Actual: [0.23489011] Predicted: [0.26651016]
[1.87] [2.100194]
Actual: [0.22527473] Predicted: [0.26371863]
[1.8] [2.0798717]
Actual: [0.23489011] Predicted: [0.2668844]
[1.87] [2.1029184]
Actual: [0.23214286] Predicted: [0.26967722]
[1.85] [2.1232502]
Actual: [0.22802198] Predicted: [0.26967916]
[1.82] [2.1232643]
Actual: [0.23076923] Predicted: [0.2669311]
[1.84] [2.1032584]
Actual: [0.22802198] Predicted: [0.2684203]
[1.82] [2.1141]
